# Basic Streams Sample

The basic cells needed are included below.  Make any changes and add your sources, analytics and outputs.

In this notebook, you'll see examples of how to:
 1. [Setup your Streams instance](#setup)
 1. [Create the application](#create)
 1. [Submit the application](#launch)
 1. [Connect to the running application to view data](#view)
 1. [Stop the application](#cancel)

# Overview

**Introduction**

This sample shows how to create and launch an application.

**How it works**
   
The Python application created in this notebook is submitted to the IBM Streams service for execution.

<img src="https://developer.ibm.com/streamsdev/wp-content/uploads/sites/15/2019/04/how-it-works.jpg" alt="How it works">


## Documentation
    
- [Streams Python development guide](https://ibmstreams.github.io/streamsx.documentation/docs/latest/python/)
- [Streams Python API](https://streamsxtopology.readthedocs.io/)



## <a name="setup"> </a> 1. Setup

### 1.1 Add credentials for the IBM Streams service

In order to submit a Streams application you need to provide the name of the Streams instance.

1. From the navigation menu, click **My instances**.
2. Click the **Provisioned Instances** tab.
3. Update the value of `streams_instance_name` in the cell below according to your Streams instance name.


In [1]:
from icpd_core import icpd_util
streams_instance_name = "cp4d-streams-instance" ## Change this to Streams instance
cfg=icpd_util.get_service_instance_details(name=streams_instance_name)
cfg

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json', 'Date': 'Fri, 13 Mar 2020 05:33:36 GMT', 'Content-Length': '108'})
HTTP response body: {"_messageCode_":"Bad Request","_statusCode_":400,"exception":"InvalidQueryParams","message":"Bad Request"}



### 1.2 Import  the `streamsx` package and verify the package version

In [ ]:
import streamsx.topology.context
print("INFO: streamsx package version: " + streamsx.topology.context.__version__)

#For more details uncomment line below.
#!pip show streamsx

<a id="create"></a>
# 2. Create the application

In [ ]:
#create topology...change the name to something more meaningful
from streamsx.topology.topology import Topology
import streamsx.topology.context

topo = Topology(name="BasicTemplate")

In [ ]:
# add stream...delete and replace this with your streams and analytics
# this is added so the topology will run (needs a source)
# produces a random integer between 1 and 100 every tenth of a second
import random, time
def readings():
    while True:
        time.sleep(0.1)
        yield random.randint(1,100)

source = topo.source(readings)

# create a view to watch the stream data while running
source_view = source.view(name="RandomIntegers", description="Random integers from source stream")

<a id="launch"></a>
# 3. Submit the application
A running Streams application is called a *job*. By submitting the topology we create a job.

In [ ]:
from streamsx.topology import context
from streamsx.topology.context import ContextTypes

# Disable SSL certificate verification if necessary
cfg[context.ConfigParams.SSL_VERIFY] = False
# submit the topology 'topo'
submission_result = context.submit (ContextTypes.DISTRIBUTED, topo, config = cfg)
submission_result
# The submission_result object contains information about the running application, or job
if submission_result.job:
    streams_job = submission_result.job
    print ("JobId: ", streams_job.id , "\nJob name: ", streams_job.name)

<a name="view"></a>
# 4. Use a `View` to access data from the job
Now that the job is started, use the `View` object to start retrieving data from a `Stream`.

In [ ]:
#show 30 seconds of data
if submission_result.job:
    queue = source_view.start_data_fetch()
    timeSpan = time.time() + 30;
    while time.time() < timeSpan:
        print(queue.get())
    source_view.stop_data_fetch()



<a name="cancel"></a>

# 5. Cancel the job

This cell generates widgets you can use to cancel the jobs.

In [ ]:
submission_result.cancel_job_button()

You can also interact with the job through the [Job](https://streamsxtopology.readthedocs.io/en/stable/streamsx.rest_primitives.html#streamsx.rest_primitives.Job) object returned from `submission_result.job`

For example, use `streams_job.cancel()` to cancel the running job directly.